In [3]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark
#import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

#%pip install pyspark
#%pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 47 kB/s 
     |████████████████████████████████| 199 kB 53.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=29d1fa9c181b5516407b13bf0e5b78c556868323ad59d0314cdd5bece1460daf
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [231]:
from pandas.core.arrays import interval
import pyspark
from pyspark import SQLContext
from pyspark.sql import SparkSession
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row
#import findspark
#findspark.init('D:\Programs\Python3.10\Lib\site-packages\pyspark')
#findspark.find()

conf = pyspark.SparkConf()
#print(conf)
conf.set("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")
conf.set("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")
conf.set("spark.driver.memory", "4g")
#sc = SparkSession.builder.getOrCreate()
#sc = SparkSession.builder.appName("Test").getOrCreate()
#sc = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkSession.builder.master("local[*]").appName("pyspark").config(conf=conf).getOrCreate()
#print(sc)
#print(sc.version)


#1. Создайте схему будущего фрейма данных. Схема должна включать следующие атрибуты:
#id -  уникальный идентификатор посетителя
#timestamp – дата и время события в формате timestamp
#type – тип события, значение из списка: посещение(visit), клик(click), скролл(scroll), переход(move).
#page_id – id текущей страницы. Тип - последовательность чисел
#tag - тематика конкретной новости: политика, спорт, медицина 
#sign – наличие у пользователя личного кабинета True/False
#Типы данных::
#(DataType, NullType, StringType, BinaryType, BooleanType, DateType, TimestampType, DecimalType, DoubleType, FloatType, ByteType, 
#IntegerType, LongType, DayTimeIntervalType, Row, ShortType, ArrayType, MapType, StructField, StructType)
from pyspark.sql.types import *
from pyspark.sql.functions import * #unix_timestamp()
schemaDF = StructType([ StructField("id", IntegerType(), False), 
                        StructField("timestamp", LongType(), False),
                        StructField("type", StringType(), False),
                        StructField("page_id", IntegerType(), False),
                        StructField("tag", StringType(), False),
                        StructField("sign", BooleanType(), False)
                       #,StructField("timeRange", DayTimeIntervalType(), False)
                        ])

from typing import Optional
#2. Создайте датафрейм с описанной выше схемой данных 
#3. Наполните датафрейм данными. Пример: (12345, 1667627426, "click", 101, "Sport”, False)
data = [(1, 1669545967, "click", 10, "Sport", False ),
         #, 4),
    (2, 1666781167, "visit", 11, "Medicine", True  ),
     #, 4),
    (3, 1669373167, "click", 101, "Medicine", False  ),
     #, 8),
    (4, 1669373167, "scroll", 103, "Politics", True  ),
    #, 12),
    (5, 1669545967, "move", 108, "Politics", False  ),
     #, 16)
  ]
dataDF = sc.createDataFrame(data=data,schema=schemaDF)
print("\n Схема датафрейма:: ")
dataDF.printSchema()
#dataDF.fieldNames()
#dataDF.show()
#dataDF.columns
#dataDF.count()
#dataDF.printSchema()


#4 Вывести топ-5 самых активных посетителей сайта
#dataDF.show(5)
print("\n Топ-5 самых активных посетителей сайта:: ")
active = dataDF.select(['id', 'timestamp']).groupBy('id').max('timestamp').show(5)


#5 Посчитать процент посетителей, у которых есть ЛК
all = dataDF.select(['sign']).count()
cur = dataDF.filter(dataDF.sign == True).count()
import math as mat
print("\n Процент посетителей, у которых есть ЛК:: ")
print(""+str(float(mat.floor((cur*100)/all)))+"% ")


#5 Вывести топ-5 страниц сайта по показателю общего кол-ва кликов на данной странице
print("\n Топ-5 страниц сайта по показателю общего кол-ва кликов:: ")
clickp = dataDF.filter(dataDF.type == "click").groupBy("page_id").agg(count("type").alias("кол-ва кликов")).show(5)


#6 Добавьте столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)
#from pyspark.sql import Window
#w=Window.orderBy("year").rowsBetween(-1,0).
print("\n Добавьте столбец к фрейму данных со значением временного диапазона:: ")
#dataDF.show(truncate=False)
newcol = dataDF.withColumn("timeRange", when(col("page_id") <= 100, 
                                  lit("4").cast(IntegerType())).otherwise(lit("12").cast(IntegerType())))
newcol.show(truncate=False)


#7 Выведите временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.
print("\n Выведите временной промежуток в течение которого было больше всего активностей:: ")
newcol.groupBy("timeRange").agg(count("timeRange").alias("Всего активностей")).orderBy(desc("timeRange")).show(5)


#8 Создайте второй фрейм данных, который будет содержать информацию о ЛК посетителя сайта со следующим списком атрибутов
#Id – уникальный идентификатор личного кабинета
#User_id – уникальный идентификатор посетителя
#ФИО посетителя
#Дату рождения посетителя 
#Дата создания ЛК
from datetime import datetime
import time
import random
random.seed(int(time.time()))
schemaDF2 = StructType([ StructField("id", IntegerType(), False),
                        StructField("User_id", IntegerType(), False),
                        StructField("FIO", StringType(), False)
                        #,StructField("UserBD", DateType(), False)
                        #,StructField("DateLK", DateType(), False)
                        ])
data2 = [(1, 1, "Ivanov I.I." 
        ),(2, 2, "Petrov P.P." 
        ),(3, 3, "Sidorov S.S." 
        ),(4, 4, "Pysparkov P.S."
        ),(5, 5, "Petuhov P.H."
        )
       ]

print("\n Создайте второй фрейм данных, который будет содержать информацию о ЛК посетителя:: ")
dataDF2 = sc.createDataFrame(data=data2,schema=schemaDF2)
dataDF2 = dataDF2.withColumn("UserBD",lit(datetime.fromtimestamp(random.randint(1, int(time.time()))).strftime('%Y-%m-%d')).cast(DateType()))
dataDF2 = dataDF2.withColumn("DateLK",lit(datetime.fromtimestamp(random.randint(1, int(time.time()))).strftime('%Y-%m-%d')).cast(DateType()))
dataDF2.printSchema()
dataDF2.show()


#9 Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт.
print("\n Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт:: ")
JoinDF = dataDF.join(dataDF2, (dataDF.id == dataDF2.User_id) & (dataDF.tag == "Sport") ,'inner')
JoinDF.select(['FIO','tag']).show()







#*************************************#
# Загрузка файла с локального диска
#from google.colab import files
#files.upload()
#from google.colab import drive
#drive.mount('/content/drive')
#*************************************#


 Схема датафрейма:: 
root
 |-- id: integer (nullable = false)
 |-- timestamp: long (nullable = false)
 |-- type: string (nullable = false)
 |-- page_id: integer (nullable = false)
 |-- tag: string (nullable = false)
 |-- sign: boolean (nullable = false)


 Топ-5 самых активных посетителей сайта:: 
+---+--------------+
| id|max(timestamp)|
+---+--------------+
|  1|    1669545967|
|  2|    1666781167|
|  3|    1669373167|
|  5|    1669545967|
|  4|    1669373167|
+---+--------------+


 Процент посетителей, у которых есть ЛК:: 
40.0% 

 Топ-5 страниц сайта по показателю общего кол-ва кликов:: 
+-------+-------------+
|page_id|кол-ва кликов|
+-------+-------------+
|     10|            1|
|    101|            1|
+-------+-------------+


 Добавьте столбец к фрейму данных со значением временного диапазона:: 
+---+----------+------+-------+--------+-----+---------+
|id |timestamp |type  |page_id|tag     |sign |timeRange|
+---+----------+------+-------+--------+-----+---------+
|1  |166954